In [0]:
from dataclasses import dataclass
from torch.utils.data import Dataset, DataLoader
dataclass(eq=False)

<function dataclasses.dataclass.<locals>.wrap>

In [0]:
@dataclass(eq=False)
class FizBuzDataset(Dataset):
    input_size: int = 10
    start: int = 0
    end: int = 1000

    def encoder(self, num):
        ret = [int(i) for i in '{0:b}'.format(num)]
        return [0] * (self.input_size - len(ret)) + ret

    def __getitem__(self, idx):
        idx += self.start
        x = self.encoder(idx)
        if idx % 15 == 0:
            y = [1, 0, 0, 0]
        elif idx % 5 == 0:
            y = [0, 1, 0, 0]
        elif idx % 3 == 0:
            y = [0, 0, 1, 0]
        else:
            y = [0, 0, 0, 1]
        return x, y

    def __len__(self):
        return self.end - self.start

In [0]:
if __name__ == '__main__':

    dataset = FizBuzDataset()
    for i in range(len(dataset)):
        x, y = dataset[i]


    dataloader = DataLoader(dataset, batch_size=10, shuffle=True, num_workers=4)
    for batch in dataloader:
        print(batch)

[[tensor([1, 0, 1, 1, 0, 0, 1, 1, 0, 0]), tensor([1, 0, 0, 1, 1, 1, 0, 1, 0, 1]), tensor([0, 1, 1, 0, 0, 1, 0, 1, 1, 0]), tensor([0, 0, 1, 1, 0, 0, 0, 0, 0, 0]), tensor([0, 0, 1, 1, 1, 0, 1, 0, 0, 1]), tensor([1, 0, 1, 1, 1, 1, 1, 0, 0, 1]), tensor([0, 0, 1, 1, 1, 1, 1, 0, 0, 0]), tensor([1, 0, 0, 1, 0, 0, 1, 1, 1, 1]), tensor([1, 1, 0, 1, 1, 1, 0, 1, 1, 0]), tensor([1, 1, 0, 0, 0, 1, 1, 0, 0, 1])], [tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 0, 0, 1, 0, 1, 1, 0, 0, 1]), tensor([1, 1, 0, 0, 1, 0, 0, 1, 1, 0])]]
[[tensor([0, 0, 1, 0, 0, 1, 0, 0, 1, 0]), tensor([1, 1, 0, 0, 0, 1, 0, 0, 1, 0]), tensor([1, 0, 0, 0, 0, 1, 0, 1, 0, 0]), tensor([0, 1, 0, 0, 1, 0, 0, 1, 0, 0]), tensor([1, 1, 0, 1, 0, 0, 0, 1, 1, 0]), tensor([1, 0, 1, 0, 0, 0, 0, 0, 1, 0]), tensor([1, 0, 0, 1, 1, 1, 0, 0, 1, 0]), tensor([1, 0, 1, 1, 0, 1, 1, 0, 1, 0]), tensor([0, 1, 0, 1, 0, 1, 1, 0, 0, 0]), tensor([0, 0, 1, 0, 0, 0, 0, 1, 0, 1])], [tensor([0, 0, 0, 0, 0, 0, 0, 1,

In [0]:
import os

import torch
import torchtext
from torchtext import data
from torchtext import datasets
import spacy
from torchtext.data import TabularDataset

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [7]:
%cd '/content/gdrive/My Drive'

/content/gdrive/My Drive


In [8]:
!ls -al

total 67
drwx------ 2 root root  4096 Sep 17 08:10 'Colab Notebooks'
-rw------- 1 root root 28772 Dec 12 05:03  ModelImplementation.ipynb
-rw------- 1 root root 14200 Sep 17 08:37  simple_model.h5
-rw------- 1 root root 10512 Sep 17 08:36  simple_weights.h5
-rw------- 1 root root  9379 Dec 12 06:10  TrainingAndValidation.ipynb


In [0]:
def downloadTREC(path='.'):
    TEXT = torchtext.data.Field()
    LABEL = torchtext.data.Field()
    if not os.path.exists('somedummyfile'):
        f = open('somedummyfile', 'w+')
        f.close()
    dataset = torchtext.datasets.TREC('somedummyfile', TEXT, LABEL)
    dataset.download(path)

In [16]:
downloadTREC()

downloading train_5500.label


train_5500.label: 100%|██████████| 336k/336k [00:00<00:00, 449kB/s]


downloading TREC_10.label


TREC_10.label: 100%|██████████| 23.4k/23.4k [00:00<00:00, 127kB/s] 


In [18]:
!ls -al trec

total 351
-rw------- 1 root root 335858 Dec 12 06:32 train_5500.label
-rw------- 1 root root  23354 Dec 12 06:32 TREC_10.label


In [0]:
spacy_en = spacy.load('en')

In [0]:
def tokenizer(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [0]:
TEXT = data.Field(sequential=True, tokenize=tokenizer, lower=True)
LABEL = data.Field(sequential=False, use_vocab=True)

In [34]:
train, val, test = TabularDataset.splits(
    path='./data/', train='TRECtrain.tsv',
    validation='TRECval.tsv', test='TRECtest.tsv', format='tsv',
    fields=[('text', TEXT), ('label', LABEL)])

FileNotFoundError: ignored

In [0]:
TEXT.build_vocab(train, vectors="glove.6B.50d")
LABEL.build_vocab(train, vectors="glove.6B.50d")
train_iter, val_iter, test_iter = data.Iterator.splits(
    (train, val, test), sort_key=lambda x: len(x.Text),
    batch_sizes=(32, 99, 99), device=-1)

NameError: ignored

In [0]:
print(next(iter(test_iter)))

NameError: ignored

In [0]:
class DummyNN(torch.nn.Module):

    def __init__(self, emb_dim):
        self.embed = torch.nn.Embedding(len(TEXT.vocab), emb_dim)
        self.embed.weight.data.copy_(TEXT.vocab.vectors)